**Recap**, we're classifying gravitational-wave sources based on their LIGO detectability. I actually tackled this problem using a multi-layer perceptron in a research paper: 

*Gravitational-wave selection effects using neural-network classifiers.*
Davide Gerosa, Geraint Pratten, Alberto Vecchio.
Physical Review D 102 (2020) 103020.
https://arxiv.org/abs/2007.06585. 


The same problem was then tackled a few years later by another group with a similar neural network but a more sophisticated (hence realistic) training dataset:

*Neural network emulator of the Advanced LIGO and Advanced Virgo selection function.*
Thomas Callister, Reed Essick, Daniele Holz 
Physical Review D 110 (2024) 123041
https://arxiv.org/abs/2408.16828

Here is the code I used, which implemented using TensorFlow. Also distributed open source here: https://github.com/dgerosa/pdetclassifier

In [11]:
import deepdish
import os
import tensorflow as tf
from tensorflow import keras
import numpy as np


def sperical_to_cartesian(mag,theta,phi):
    '''
    Convert spherical to cartesian coordinates
    '''
    coordx = mag * np.cos(phi) * np.sin(theta)
    coordy = mag * np.sin(phi) * np.sin(theta)
    coordz = mag * np.cos(theta)
    return coordx,coordy,coordz

massvar = ['mtot','q','z']
spinvar = ['chi1x','chi1y','chi1z','chi2x','chi2y','chi2z']
intrvar=massvar+spinvar
extrvar = ['iota','ra','dec','psi']
train_variables=intrvar+extrvar

def lookup_limits():
    '''
    Define the limits in all variables. If you want to change this, please check generate_binaries() and pdet() as well.
    '''

    limits={
        'mtot'  : [2,1000],
        'q'     : [0.1,1],
        'z'     : [1e-4,4],
        'chi1x'  : [-1,1],
        'chi1y'  : [-1,1],
        'chi1z'  : [-1,1],
        'chi2x'  : [-1,1],
        'chi2y'  : [-1,1],
        'chi2z'  : [-1,1],
        'iota'  : [0,np.pi],
        'ra'    : [-np.pi,np.pi],
        'dec'   : [-np.pi/2,np.pi/2],
        'psi'   : [0,np.pi]
        }

    return limits


def store_binaries(filename, N, approximant='IMRPhenomXPHM', noisecurve='design', SNRthreshold=12):
    ''' Generate binaries, compute SNR, and store'''

    inbinaries = generate_binaries(N)
    outbinaries = evaluate_binaries(inbinaries, approximant, noisecurve, SNRthreshold)

    deepdish.io.save(filename,outbinaries)
    return filename

def readsample(filename='sample.h5'):
    '''
    Read a validation sample that already exists
    '''
    return deepdish.io.load(filename)

def splittwo(binaries):
    '''
    Split sample into two subsamples of equal size
    '''

    one={}
    two={}
    for k in train_variables+['snr','det']:
        one[k],two[k] = np.split(binaries[k],2)
    one['N'],two['N']= len(one['mtot']),len(two['mtot'])

    return one,two


def rescale(x,var):
    '''
    Rescale variable sample x of variable var between -1 and 1
    '''

    limits=lookup_limits()
    if var not in limits:
        raise ValueError

    return 1-2*(np.array(x)-min(limits[var]))/(max(limits[var])-min(limits[var]))


def nnet_in(binaries):
    '''
    Prepare neural network inputs.
    '''

    return np.array([rescale(binaries[k],k) for k in train_variables]).T

def nnet_out(binaries, which='detnetwork'):
    '''
    Prepare neural network outputs.
    '''

    return binaries['det']


def trainnetwork(train_binaries,test_binaries,filename='trained.h5'):

    if not os.path.isfile(filename):

        train_in  = nnet_in(train_binaries)
        train_out = nnet_out(train_binaries)
        test_in  = nnet_in(test_binaries)
        test_out = nnet_out(test_binaries)

        # Kernel initializer
        my_init = keras.initializers.glorot_uniform(seed=1)
        # Define neural network architecture
        model = keras.Sequential([
            # Input layer, do not change
            tf.keras.layers.InputLayer(input_shape=np.shape(train_in[0])),
            # Inner layers, can add/change
            keras.layers.Dense(32,  activation='tanh',kernel_initializer=my_init),
            #keras.layers.Dense(16,  activation='tanh',kernel_initializer=my_init),
            #keras.layers.Dense(8,  activation='tanh',kernel_initializer=my_init),
            # Output layer, do not change
            keras.layers.Dense(1, activation='sigmoid',kernel_initializer=my_init)])

        model.compile(
            # Optimization algorithm, specify learning rate
            
            #You should use this one...
            optimizer=keras.optimizers.Adam(learning_rate=1e-2),
            #... but it's not optimized for the new Apple M2 chips, so I'm using an older one:
            # optimizer=keras.optimizers.legacy.Adam(learning_rate=1e-2),

            # Loss function for a binary classifier
            loss='binary_crossentropy',
            # Diagnostic quantities
            metrics=['accuracy'])

        # Decrease the learning rate exponentially after the first 10 epochs
        def scheduler(epoch, lr):
            if epoch < 10:
                return lr
            else:
                return lr * tf.math.exp(-0.05)

        # Actual Training
        history = model.fit(
            # Training inputs
            train_in,
            # Training outputs
            train_out,
            # Evaluate test set at each epoch
            validation_data=(test_in, test_out),
            # Batch size, default is 32
            #batch_size=32,
            # Number of epochs
            epochs=150,
            # Store the model with the best validation accuracy
            callbacks = [
                # Drecrease learning rate
                tf.keras.callbacks.LearningRateScheduler(scheduler),
                # Store the model with the best validation accuracy
                tf.keras.callbacks.ModelCheckpoint(
                    filepath=filename,
                    save_weights_only=False,
                    monitor='val_accuracy',
                    mode='max',
                    save_best_only=True),
                # Save logfiles for tensorboard
                tf.keras.callbacks.TensorBoard(log_dir="logs"+filename.split('.h5')[0], histogram_freq=1)],
            # Shuffle data at each epoch
            shuffle=True)

        # Store the last (not necessarily the best) iteration
        #model.save(filename)

    model = loadnetwork(filename)
    return model


def loadnetwork(filename,verbose=False):
    '''
    Load a trained neural network
    '''

    model = tf.keras.models.load_model(filename)
    if verbose:
        model.summary()

    return model


def testnetwork(model,binaries):
   '''
   Test network on a series of binaries
   '''
   test_in  = nnet_in(binaries)
   test_out = nnet_out(binaries)
   model.evaluate(test_in,  test_out, verbose=2)


def predictnetwork(model, binaries):
    '''
    Use a network to predict the detectability of a set of binaries.
    '''
    # Return the class (0 or 1) that is preferred
    predictions = np.squeeze((model.predict(nnet_in(binaries)) > 0.5).astype("int32"))
    return predictions


In [12]:
binaries= readsample('sample_2e7_design_precessing_higherordermodes_3detectors.h5')

In [13]:
# Split test/training
train_binaries,test_binaries=splittwo(binaries)

In [14]:
# Train a neural network
trainnetwork(train_binaries,test_binaries,filename='trained.h5')

Epoch 1/150
  6294/312500 [..............................] - ETA: 1:27 - loss: 0.0963 - accuracy: 0.9582

KeyboardInterrupt: 

That's going to take ~12 hours on this laptopt (I don't have a GPU turned on...). Here is the trained file:
https://github.com/dgerosa/pdetclassifier/blob/master/trained_2e7_design_precessing_higherordermodes_3detectors.h5


In [16]:
import requests
url = 'https://github.com/dgerosa/pdetclassifier/blob/master/trained_2e7_design_precessing_higherordermodes_3detectors.h5?raw=true'
r = requests.get(url, allow_redirects=True)
open('trained_2e7_design_precessing_higherordermodes_3detectors.h5', 'wb').write(r.content)

29608

In [17]:
import tensorflow as tf

# Load trained network
model = loadnetwork('trained_2e7_design_precessing_higherordermodes_3detectors.h5')

In [18]:
# Some downsampling, otherwise the evaluation takes to long for class
test_less={}
for k in ['mtot', 'q', 'z', 'chi1x', 'chi1y', 'chi1z', 'chi2x', 'chi2y', 'chi2z', 'iota', 'ra', 'dec', 'psi', 'snr', 'det']:
    test_less[k]=test_binaries[k][:100]

In [19]:
testnetwork(model,test_less)

4/4 - 0s - loss: 0.0682 - accuracy: 0.9800 - 56ms/epoch - 14ms/step


In [20]:
predictnetwork(model, test_less)

4/4 [==============================] - 0s 581us/step


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0], dtype=int32)